
## Purpose
Pre-process the output from the GEE data match-ups prior to analysis


## Library import
Rquired Python libraries

In [1]:
# Data manipulation
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dateutil.parser import parse

from pathlib import Path

# Parameter definition
- Input path where the GEE output is saved . 

In [2]:
# '.' means current working directory
file = Path('.').joinpath('ToyamaBay_Bottle_update2022184_3h_0.5+2m.csv')

- Load the date and display the info

In [3]:
df = pd.read_csv(file)
df.mask(df==-999, inplace=True)
df

,system:index,Chla,Rrs412,Rrs443,Rrs490,Rrs530,Rrs565,Rrs670,ay412,ay440,ss,y,.geo
0,00000000000000000b01_GC1SG1_201801180050004200...,1.172,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1283,4.28,"{chla=null, cdom=null, tsm=null, qa_flag=4101.0}","{""type"":""Point"",""coordinates"":[137.02565152119..."
1,00000000000000000b01_GC1SG1_201801180050004200...,1.172,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1283,4.28,"{Rrs_380=null, Rrs_412=null, Rrs_443=null, Rrs...","{""type"":""Point"",""coordinates"":[137.02565152119..."
2,00000000000000000b01_GC1SG1_201801180231007600...,1.172,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1283,4.28,"{chla=null, cdom=null, tsm=null, qa_flag=65534.0}","{""type"":""Point"",""coordinates"":[137.02565152119..."
3,00000000000000000b01_GC1SG1_201801180231007600...,1.172,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1283,4.28,"{Rrs_380=null, Rrs_412=null, Rrs_443=null, Rrs...","{""type"":""Point"",""coordinates"":[137.02565152119..."
4,00000000000000000b02_GC1SG1_201801180050004200...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{chla=null, cdom=null, tsm=null, qa_flag=4101.0}","{""type"":""Point"",""coordinates"":[137.02565152119..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
859,00000000000000000ed6_GC1SG1_202201260113004700...,0.991,NaN,NaN,NaN,NaN,NaN,NaN,0.0252,0.0061,0.16,"{Rrs_380=0.0030270004644989967, Rrs_412=0.0012...","{""type"":""Point"",""coordinates"":[137.25332430278..."
860,00000000000000000ed6_GC1SG1_202201260250008100...,0.991,NaN,NaN,NaN,NaN,NaN,NaN,0.0252,0.0061,0.16,"{chla=null, cdom=null, tsm=null, qa_flag=65534.0}","{""type"":""Point"",""coordinates"":[137.25332430278..."
861,00000000000000000ed6_GC1SG1_202201260250008100...,0.991,NaN,NaN,NaN,NaN,NaN,NaN,0.0252,0.0061,0.16,"{Rrs_380=null, Rrs_412=null, Rrs_443=null, Rrs...","{""type"":""Point"",""coordinates"":[137.25332430278..."
862,00000000000000000ed6_GC1SG1_202201260255008100...,0.991,NaN,NaN,NaN,NaN,NaN,NaN,0.0252,0.0061,0.16,"{chla=null, cdom=null, tsm=null, qa_flag=65534.0}","{""type"":""Point"",""coordinates"":[137.25332430278..."


- y has both ['chla', 'cdom', 'tsm'] and ['Rrs_...']
- So we convert the values from dictionaries to corresponding rows in the dataframe. 
- We also add the flag information. Currently it is encoded as a number.

In [4]:
# convert the string output from GEE to dictionary
def to_dict(data):
    return eval(f'dict({data.y[1:-1]})')

In [5]:
# We have IWPR flags and NWLR flags.
# Associate each datatype to its variables
def append_dtype(keys):
    if 'chla' in keys:
        return [f'{key}_iwp' for key in keys]
    return [f'{key}_nlw' for key in keys]

In [6]:
# new data columns to be added to the dataframe
columns, null = [], None
for i, row in df.iterrows():
    if i > 1:
        break
    row = to_dict(row)
    columns.extend(
        append_dtype(row.keys())
    )
columns

['chla_iwp',
 'cdom_iwp',
 'tsm_iwp',
 'qa_flag_iwp',
 'Rrs_380_nlw',
 'Rrs_412_nlw',
 'Rrs_443_nlw',
 'Rrs_490_nlw',
 'Rrs_530_nlw',
 'Rrs_565_nlw',
 'Rrs_670_nlw',
 'qa_flag_nlw']

### The NWLR flags have different sequence and order from IWPR.
I created the short names from the long-names contained in the files as follows. The IWPR flags follow the standard flag naming as in GPortal.

```Python
'no observation data': 'NODATA',
'land pixel': 'LAND',
'incomplete VNR bands': 'DATAMISS',
'cloud or ice': 'CLDICE',
'near cloud (+-2pix)': 'CLDAFFCTD',
'straylight flag': 'STRAYLIGHT',
'sunglint mask>0.16': 'HIGLINT',
'sunglint flag>0.01': 'MODGLINT',
'soz>75': 'HISOLZ',
'taua>0.5': 'HITAUA',
'negative nLw': 'NEGNLW',
'turbid Case-2 water': 'TURBIDW',
'coast pixel': 'COASTZ',
'dark pixel': 'LOWNLW',
'out of aerosol models': 'PRODFAIL',
'wind speed>20m/s': 'HIWNDSPD'
 ```

In [7]:
flags = {
    'IWPR': {0: "DATAMISS", 1: "LAND", 2: "ATMFAIL", 3: "CLDICE", 4: "CLDAFFCTD",
             5: "STRAYLIGHT", 6: "HIGLINT", 7: "MODGLINT", 8: "HISOLZ", 9: "HITAUA",
             10: "NEGNLW", 11: "ATM-METHOD", 12: "SHALLOW", 13: "ITERFAILCDOM", 
             14: "CHLWARN", 15: "SPARE"},
    
    'NWLR': {0: "NODATA", 1: "LAND", 2: "DATAMISS", 3: "CLDICE", 4: "CLDAFFCTD",
             5: "LOWNLW", 6: "COASTZ", 7: "STRAYLIGHT", 8: "HIGLINT", 9: "MODGLINT",
             10: "HIWNDSPD", 11: "HISOLZ", 12: "HITAUA", 13: "PRODFAIL", 14: "NEGNLW", 
             15: "TURBIDW"}
}

In [8]:
def qa_flag_meaning(flag, key):
    found = np.zeros((16,), int)
    for bit in flags[key].keys():
        shift = (1 << bit) & flag
        found[bit] = shift > 0
    loop = zip(flags[key].values(), found)
    return '+'.join([
        flag_name * fval
        for flag_name, fval in loop
        if fval > 0
    ])

In [9]:
for i, row in df.iterrows():
    geoloc = eval(row['.geo'])['coordinates']
    df.loc[i, ['lon', 'lat']] = geoloc
    row = to_dict(row)
    
    if 'chla' in row.keys():
        key, cols = 'IWPR', columns[:4]
    else:
        key, cols = 'NWLR', columns[4:]
    row_values = list(row.values())
    row_values[-1] = qa_flag_meaning(int(row_values[-1]), key)
    df.loc[i, cols] = row_values
df = df.drop(columns=['y', '.geo'])
df

,system:index,Chla,Rrs412,Rrs443,Rrs490,Rrs530,Rrs565,Rrs670,ay412,ay440,...,tsm_iwp,qa_flag_iwp,Rrs_380_nlw,Rrs_412_nlw,Rrs_443_nlw,Rrs_490_nlw,Rrs_530_nlw,Rrs_565_nlw,Rrs_670_nlw,qa_flag_nlw
0,00000000000000000b01_GC1SG1_201801180050004200...,1.172,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1283,...,NaN,DATAMISS+ATMFAIL+SHALLOW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00000000000000000b01_GC1SG1_201801180050004200...,1.172,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1283,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NODATA+DATAMISS
2,00000000000000000b01_GC1SG1_201801180231007600...,1.172,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1283,...,NaN,LAND+ATMFAIL+CLDICE+CLDAFFCTD+STRAYLIGHT+HIGLI...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00000000000000000b01_GC1SG1_201801180231007600...,1.172,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1283,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LAND+DATAMISS+CLDICE+CLDAFFCTD+LOWNLW+COASTZ+S...
4,00000000000000000b02_GC1SG1_201801180050004200...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,DATAMISS+ATMFAIL+SHALLOW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859,00000000000000000ed6_GC1SG1_202201260113004700...,0.991,NaN,NaN,NaN,NaN,NaN,NaN,0.0252,0.0061,...,NaN,NaN,0.003027,0.001281,0.003777,0.001839,0.004638,0.000363,0.000363,CLDAFFCTD+LOWNLW+HITAUA
860,00000000000000000ed6_GC1SG1_202201260250008100...,0.991,NaN,NaN,NaN,NaN,NaN,NaN,0.0252,0.0061,...,NaN,LAND+ATMFAIL+CLDICE+CLDAFFCTD+STRAYLIGHT+HIGLI...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
861,00000000000000000ed6_GC1SG1_202201260250008100...,0.991,NaN,NaN,NaN,NaN,NaN,NaN,0.0252,0.0061,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LAND+DATAMISS+CLDICE+CLDAFFCTD+LOWNLW+COASTZ+S...
862,00000000000000000ed6_GC1SG1_202201260255008100...,0.991,NaN,NaN,NaN,NaN,NaN,NaN,0.0252,0.0061,...,NaN,LAND+ATMFAIL+CLDICE+CLDAFFCTD+STRAYLIGHT+HIGLI...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Export the formatted table
df.to_csv(
    file.parent.joinpath(
        file.name.replace(file.suffix, '_formatted.csv')
    ), index=False
)